<a href="https://colab.research.google.com/github/CristopherCano/Proyecto_Python_Procesamiento_de_datos/blob/main/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Procesamiento de Datos con Python - BEDU - Módulo 3
## Viajes en Taxis y Ubers en la Zona Metropolitana del Valle de México 2016 - 2017
* Eduardo Alán Hernández Villasana
* Cristopher Arvizu Cano
* David Gil Peralta
* José Antonio Aguilar Téllez

## Planteamiento del Problema
En los últimos años, desde la llegada de aplicaciones para el transporte privado como Uber o Didi, los gremios taxistas de la Zona Metropolitana del Valle de México han protestado en contra de estas aplicaciones, ellos argumentan una competencia desleal y exigen regulaciones hacia estos transportes. Esto no es de extrañarse ya que, con el aumento de la popularidad de estas aplicaciones de transporte, los taxistas están perdiendo clientes y no pueden o quieren modernizarse para poder competir con los automóviles de Uber o Didi que son más modernos. 

A raíz de este enfrentamiento, han surgido noticias comparando estos dos modelos de transporte privado, sin embargo, muchas de estas noticias están sesgadas por intereses políticos y económicos. Es por esta razón que decidimos analizar datos recopilados tanto en Taxis y Ubers para hacer una comparación entre los dos modelos y aportar nuestros hallazgos estadísticos a la discusión de este problema.

## Preguntas Clave
Durante el desarrollo de este proyecto, se espera responder las siguientes preguntas.

* ¿Qué tipo de transporte pasa menos tiempo en el tráfico?
* ¿Cuál es la velocidad promedio de cada tipo de transporte?
* ¿Qué día de la semana hay mas demanda para cada tipo de transporte?
* ¿En qué tipo de transporte recibe más dinero en promedio por viaje?
* ¿Desde qué demsarcación territorial o municipio cada tipo de transporte tiene mayor demanda?
* ¿Cuál es el destino mas solicitado para cada tipo de transporte?

In [1]:
# Librerías a usar
import pandas as pd
import numpy as np

## Extracción de datos
Se descargará la base de datos desde GitHub.

In [2]:
url = "https://raw.githubusercontent.com/CristopherCano/Proyecto_Python_Procesamiento_de_datos/main/Data/cdmx_transporte_raw.csv"

In [3]:
data_raw = pd.read_csv(url, index_col=0)
data_raw.head(2)

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,,
1,México DF Taxi de Sitio,2016-09-16 07:14:12,2016-09-18 04:41:40,-99.097369,19.416874,-99.202729,19.430353,N,120449,12373,242
2,México DF Taxi Libre,2016-09-18 06:16:33,2016-09-18 10:11:43,-99.297148,19.322128,-99.289949,19.326538,N,14110,1700,461


### Descripción de las Columnas
* **id**: Un identificador único para cada viaje.
* **vendor_id**: El tipo de transporte en que se realizó el viaje.
* **pickup_datetime**: Fecha y Hora en que el viaje inició.
* **dropoff_datetime**: Fecha y Hora en que el viaje finalizó.
* **pickup_longitude**: Longitud en la que el viaje inició.
* **pickup_latitude**: Latitud en la que el viaje inició.
* **dropoff_longitude**: Longitud en la que el viaje finalizó.
* **dropoff_latitude**: Latitud en la que el viaje finalizó.
* **store_and_fwd_flag**: Indica si la información del viaje se almacenó en una memoria y después se envio al servidor por que no tenía conexión a internet o no. Todas las entradas de datos estan en 'N'.
* **trip_duration**: Duración del viaje en segundos.
* **dist_meters**: Distancia recorrida en el viaje en metros.
* **wait_sec**: Tiempo en segundos en el que el automovil estuvo completamente detenido durante el viaje. (Se usará como medición del tráfico.)

## Exploración básica de datos
### Hallazgos preliminares encontraros 

1. ¿El conjunto de datos que tengo realmente me sirve para responder algunas de las preguntas que me planteé?

In [4]:
tamaño = data_raw.shape
print(f'2. ¿Qué tamaño tiene mi conjunto de datos? \n número de filas: {tamaño[0]} \n número de columnas: {tamaño[1]}')

2. ¿Qué tamaño tiene mi conjunto de datos? 
 número de filas: 12694 
 número de columnas: 11


In [5]:
print(f'3. ¿Qué columnas tengo y qué información tengo en cada una de esas columnas? \n \n En total tenemos{tamaño[1]} columnas \n \n {data_raw.columns}')

3. ¿Qué columnas tengo y qué información tengo en cada una de esas columnas? 
 
 En total tenemos11 columnas 
 
 Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'store_and_fwd_flag', 'trip_duration', 'dist_meters', 'wait_sec'],
      dtype='object')


4. Los nombres que tienen mis columnas, ¿son el nombre más apropiado?

Podemos observar que algunos nombres como vendor_id o dist_meters no son los más adecuados por lo que tendremos que usar la conveción apropiada que es snake-case y ajustar sus nombres

5. ¿Qué tipos de datos tengo en cada columna? ¿Parecen ser el tipo correcto de datos? ¿O es un tipo de datos "incorrecto"?



In [6]:
data_raw.dtypes

vendor_id              object
pickup_datetime        object
dropoff_datetime       object
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dist_meters             int64
wait_sec                int64
dtype: object

Las fechas aparecen como object por lo que sera necesario transformar esta variable a un formato de fecha mediante padas

6. Si selecciono algunas filas al azar y las observo, ¿estoy obteniendo los datos que debería? ¿O hay datos que parecen estar "sucios" o "incorrectos"?


In [7]:
data_raw.sample(5)

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,,
8567,México DF Taxi Libre,2017-05-18 02:34:33,2017-05-18 02:55:27,-99.195952,19.461670,-99.191805,19.461253,N,1254,4092,631
11300,México DF Taxi Libre,2017-07-03 12:31:00,2017-07-03 12:36:34,-99.196370,19.461554,-99.201323,19.454915,N,334,1448,101
9635,México DF Taxi Libre,2016-12-07 08:23:31,2016-12-07 08:35:23,-99.060312,19.307535,-99.054727,19.320659,N,712,3583,73
7086,México DF Radio Taxi,2017-01-27 03:08:14,2017-01-27 03:24:12,-99.179988,19.370164,-99.170870,19.377832,N,959,2519,138
11580,México DF Taxi Libre,2017-07-06 05:07:44,2017-07-06 05:40:52,-99.195808,19.463919,-99.191147,19.462903,N,1988,680,18


A continuación observamos los siguiente problemas:
* **vendor_id**: Se repite el patrón México DF
* **store_and_fwd_flag**: No es significativa esta variable ya que siempre marca "N" en cada fila
* **trip_duration**: Tenemos duraciones de viaje exageradamente grandes
* **dist_meters**: Tenemos distacias de viaje exageradamente grandes
* **wait_sec**: Tenemos tiempos de espera de viaje exageradamente grandes

## Limpieza de datos y agregaciones

In [8]:
# Aqui va la corrección de nombres de las columnas

In [9]:
data_raw_dropped = data_raw.drop(['store_and_fwd_flag'], axis=1)

In [10]:
 data_raw_dropped['vendor_id'] = data_raw_dropped['vendor_id'].str.replace("México DF ","")

In [11]:
data_raw_dropped.head(10)

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,
1,Taxi de Sitio,2016-09-16 07:14:12,2016-09-18 04:41:40,-99.097369,19.416874,-99.202729,19.430353,120449,12373,242
2,Taxi Libre,2016-09-18 06:16:33,2016-09-18 10:11:43,-99.297148,19.322128,-99.289949,19.326538,14110,1700,461
3,Taxi Libre,2016-09-18 10:11:50,2016-09-18 10:23:11,-99.289603,19.326263,-99.271874,19.328530,681,2848,129
4,Taxi Libre,2016-09-18 10:23:38,2016-09-18 10:30:53,-99.271161,19.328875,-99.279900,19.326256,436,1409,106
5,Taxi Libre,2016-09-18 10:44:18,2016-09-18 10:51:40,-99.282761,19.326944,-99.291705,19.322754,442,1567,85
6,Taxi Libre,2016-09-18 10:58:07,2016-09-18 10:59:46,-99.289712,19.322929,-99.289280,19.325978,100,797,19
7,Taxi Libre,2016-09-18 11:00:00,2016-09-18 11:05:45,-99.285391,19.326247,-99.282549,19.328117,345,676,169
8,Taxi Libre,2016-09-18 11:08:03,2016-09-18 11:17:06,-99.285891,19.330022,-99.289828,19.319170,544,3771,37
9,Radio Taxi,2016-09-18 01:06:08,2016-09-18 01:26:34,-99.182346,19.370885,-99.176534,19.341410,1226,5662,572


In [12]:
data_raw_dropped['pickup_datetime'] = pd.to_datetime(data_raw_dropped['pickup_datetime'], unit = 'ns')
data_raw_dropped['pickup_datetime'].head()

id
1   2016-09-16 07:14:12
2   2016-09-18 06:16:33
3   2016-09-18 10:11:50
4   2016-09-18 10:23:38
5   2016-09-18 10:44:18
Name: pickup_datetime, dtype: datetime64[ns]

In [13]:
data_raw_dropped['dropoff_datetime'] = pd.to_datetime(data_raw_dropped['dropoff_datetime'], unit = 'ns')
data_raw_dropped['dropoff_datetime'].head()

id
1   2016-09-18 04:41:40
2   2016-09-18 10:11:43
3   2016-09-18 10:23:11
4   2016-09-18 10:30:53
5   2016-09-18 10:51:40
Name: dropoff_datetime, dtype: datetime64[ns]

In [14]:
data_raw_dropped.dtypes

vendor_id                    object
pickup_datetime      datetime64[ns]
dropoff_datetime     datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
trip_duration                 int64
dist_meters                   int64
wait_sec                      int64
dtype: object

### Se han encontrado algunos viajes con velocidades, distacias o diraciones irreales, esto puede ser debido a errores en la recolección de los datos. Posteriormente se eliminatán los outliers de estas columnas.
Para hacer el filtrado más facil, se convertiran las distancias de metros a kilómetros, la duración de los viajes de segundos a horas y se deducirán las velocidades en kilómetros por hora.

In [15]:
data_filtered = data_raw_dropped.copy()

Conversión de la duración del viaje de segundos a horas.

In [16]:
data_filtered["trip_duration"].head(2)

id
1    120449
2     14110
Name: trip_duration, dtype: int64

In [17]:
# Como es una operación simple se usará una función anónima.
data_filtered["trip_duration"] = data_filtered["trip_duration"].apply(lambda x: x/3600)
data_filtered["trip_duration"].head(2)

id
1    33.458056
2     3.919444
Name: trip_duration, dtype: float64

Conversión de la distancia recorrida en metros a kilómetros

In [18]:
data_filtered["dist_meters"].head(2)

id
1    12373
2     1700
Name: dist_meters, dtype: int64

In [19]:
# Como es una operación simple se usará una función anónima.
data_filtered["dist_km"] = data_filtered["dist_meters"].apply(lambda x: x/1000)
data_filtered["dist_km"].head(2)

id
1    12.373
2     1.700
Name: dist_km, dtype: float64

Creación de la columna `speed_km` que contiene la velocidad del viaje.

In [20]:
data_filtered["speed_km"] = data_filtered["dist_km"]/data_filtered["trip_duration"]
data_filtered[["dist_km","trip_duration","speed_km"]].head()

,dist_km,trip_duration,speed_km
id,,,
1,12.373,33.458056,0.369806
2,1.700,3.919444,0.433735
3,2.848,0.189167,15.055507
4,1.409,0.121111,11.633945
5,1.567,0.122778,12.762896


In [21]:
# Se elimina la columna dist_meters
data_filtered = data_filtered.drop(["dist_meters"], axis = 1)

Como se puede ver, hay viajes que duran más de lo anormal, debido al contexto del dataframe, consideramos que se deben de eliminar los viajes con velocidades mayores a 200 km/h y con distancias menores a 500 metros (.5 km). Después se eliminarán outliers con el método del Rango Inter Cuartílico.

In [22]:
data_filtered = data_filtered[(data_filtered["speed_km"] < 200)  & (data_filtered["dist_km"] > 0.5)]
data_filtered.shape

(11411, 11)

Eliminación de Outliers de `speed_km`, `dist_km`, `trip_duration`

In [23]:
Q1 = data_filtered["speed_km"].quantile(0.25)
Q2 = data_filtered["speed_km"].quantile(0.5)
Q3 = data_filtered["speed_km"].quantile(0.75)
IQR_speed = Q3 - Q1
print ("Cuartiles\t\tValor")
print("25%\t\t",Q1)
print("50%\t\t",Q2)
print("75%\t\t",Q3)
print("\n\nRango Intercuartílico: ",IQR_speed)

Cuartiles		Valor
25%		 9.185913076398961
50%		 15.399341021416804
75%		 20.984072667115985


Rango Intercuartílico:  11.798159590717024


In [24]:
data_filtered = data_filtered[(data_filtered["speed_km"] >= Q1 - 1.5*IQR_speed) & (data_filtered["speed_km"] <= Q3 + 1.5*IQR_speed)]

In [25]:
Q1 = data_filtered["dist_km"].quantile(0.25)
Q2 = data_filtered["dist_km"].quantile(0.5)
Q3 = data_filtered["dist_km"].quantile(0.75)
IQR_dist = Q3 - Q1
print ("Cuartiles\t\tValor")
print("25%\t\t",Q1)
print("50%\t\t",Q2)
print("75%\t\t",Q3)
print("\n\nRango Intercuartílico: ",IQR_dist)

Cuartiles		Valor
25%		 2.35
50%		 4.428
75%		 8.7295


Rango Intercuartílico:  6.3795


In [26]:
data_filtered = data_filtered[(data_filtered["dist_km"] >= Q1 - 1.5*IQR_dist) & (data_filtered["dist_km"] <= Q3 + 1.5*IQR_dist)]

In [27]:
Q1 = data_filtered["trip_duration"].quantile(0.25)
Q2 = data_filtered["trip_duration"].quantile(0.5)
Q3 = data_filtered["trip_duration"].quantile(0.75)
IQR_trip_duration = Q3 - Q1
print ("Cuartiles\t\tValor")
print("25%\t\t",Q1)
print("50%\t\t",Q2)
print("75%\t\t",Q3)
print("\n\nRango Intercuartílico: ",IQR_trip_duration)

Cuartiles		Valor
25%		 0.15527777777777776
50%		 0.2936111111111111
75%		 0.587013888888889


Rango Intercuartílico:  0.4317361111111112


In [28]:
data_filtered = data_filtered[(data_filtered["trip_duration"] >= Q1 - 1.5*IQR_trip_duration) & (data_filtered["trip_duration"] <= Q3 + 1.5*IQR_trip_duration)]

In [29]:
data_filtered.shape

(8937, 11)